In [32]:
# Bibliotecas
from pdfminer.high_level import extract_text
import psycopg2
import re
import os

In [33]:
def extrair_texto_pdf(caminho_pdf):
    texto = extract_text(caminho_pdf)
    return texto

In [34]:
def salvar_questoes_discursivas(texto):
    questoes_discursivas = []
    linhas = texto.split('\n')
    i = 0

    while i < len(linhas):
        # Verifica se a linha indica o início de uma nova questão discursiva
        if re.match(r'QUEStãO diSCURSiVa \d+', linhas[i], re.IGNORECASE):
            questao = {}

            # Captura o número da questão
            questao['numero'] = linhas[i].strip()
            i += 1

            # Captura o enunciado da questão até o início da próxima questão ou fim do texto
            enunciado = []
            while i < len(linhas):
                linha = linhas[i].strip()
                
                # Verifica se chegou ao início da próxima questão discursiva ou questões objetivas
                if re.match(r'QUEStãO \d+', linha, re.IGNORECASE) or re.match(r'QUEStãO diSCURSiVa \d+', linha, re.IGNORECASE):
                    break
                
                if linha:  # Inclui a linha no enunciado se não for vazia
                    enunciado.append(linha)
                
                i += 1

            questao['enunciado'] = ' '.join(enunciado).strip()
            questoes_discursivas.append(questao)
        else:
            i += 1  # Avança para a próxima linha

    return questoes_discursivas


In [35]:
def salvar_questoes(texto):
    questoes = []
    linhas = texto.split('\n')
    i = 0

    while i < len(linhas):
        # Verifica se a linha indica o início de uma nova questão
        if re.match(r'QUEStãO \d+', linhas[i], re.IGNORECASE):
            questao = {}

            # Captura o número da questão
            questao['numero'] = linhas[i].strip()
            i += 1

            # Captura o enunciado da questão até o início das alternativas
            enunciado = []
            while i < len(linhas):
                linha = linhas[i].strip()
                
                # # Verifica se chegou às alternativas (A, B, C, D, E)
                if re.match(r'^[ABCDE] {2,}', linha):  # Alternativas formatadas como A), B), etc.
                    break
                
                if linha:  # Inclui a linha no enunciado se não for vazia
                    enunciado.append(linha)
                
                i += 1
 
            questao['enunciado'] = ' '.join(enunciado).strip()

            # Captura alternativas (A, B, C, D, E)
            alternativas = {}
            while i < len(linhas):
                linha = linhas[i].strip()
                if linha.startswith('A'):
                    alternativas['A'] = linha
                elif linha.startswith('B'):
                    alternativas['B'] = linha
                elif linha.startswith('C'):
                    alternativas['C'] = linha
                elif linha.startswith('D'):
                    alternativas['D'] = linha
                elif linha.startswith('E'):
                    alternativas['E'] = linha
                    break  # Finaliza a captura ao encontrar a alternativa E)
                i += 1

            questao['alternativas'] = alternativas
            questoes.append(questao)
        else:
            if "questionário de percepção da prova".lower() in linhas[i].lower():
                break
            i += 1  # Avança para a próxima linha

    return questoes

In [36]:
def pesquisar_materia(conexao, nome_materia):
# Pesquisa no banco de dados qual o id referente ao nome da matéria, garantindo o id correto para o relacionamento

    try:
        # Cria um cursor para executar comandos no banco de dados
        with conexao.cursor() as cursor:
            # Consulta SQL para pesquisar a matéria pelo nome
            query = f"SELECT int_cd_curso FROM curso WHERE chr_curso = '{nome_materia}'"
            
            # Executa a consulta com o parâmetro fornecido
            cursor.execute(query, (nome_materia,))
            
            # Obtém todos os resultados da consulta
            resultados = cursor.fetchall()
            
            # Verifica se há resultados
            if resultados:
                for resultado in resultados:
                    return resultado[0]
                    # print(f"{resultado[0]}")
            else:
                print("Nenhuma matéria encontrada com esse nome.")
    
    except Exception as e:
        print(f"Erro ao executar a pesquisa: {e}")

In [37]:
def conectar_banco():
    try:
        # Conecta ao banco de dados PostgreSQL
        conexao = psycopg2.connect(
        host="localhost",
        database="enadequest-provas",  # Substitua pelo nome do seu banco de dados
        user="postgres",        # Substitua pelo seu nome de usuário
        password="admin"       # Substitua pela sua senha
    )
        return conexao
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None

In [38]:
def inserir_dados(conexao, prova):
# Função para inserir dados no banco de dados
    try:
        # Cria um cursor para executar comandos SQL
        cursor = conexao.cursor()

        for item in prova:
            # Comando SQL para inserção
            sql_insert = """
            INSERT INTO prova (int_cd_curso, dat_dt_ano_aplicacao)
            VALUES (%s, %s)
            """

            # Executa o comando SQL de inserção
            cursor.execute(sql_insert, (item['id_curso'], item['ano_aplicacao']))

            # Confirma a transação
            conexao.commit()

            print("Dados inseridos com sucesso!")

    except Exception as e:
        print("Ocorreu um erro ao inserir os dados:", e)
        # Desfaz a transação em caso de erro
        conexao.rollback()
    finally:
        # Fecha o cursor
        cursor.close()


In [39]:
# Criação do Banco de dados 
def executar_script_sql(caminho_arquivo_sql, conexao):
    # Abre o arquivo SQL
    with open(caminho_arquivo_sql, 'r') as arquivo_sql:
        script_sql = arquivo_sql.read()
    
    # Cria um cursor para executar os comandos SQL
    cursor = conexao.cursor()
    
    try:
        # Executa o script SQL
        cursor.execute(script_sql)
        # Confirma as mudanças no banco de dados
        conexao.commit()
        print("Script SQL executado com sucesso.")
    except Exception as e:
        # Em caso de erro, desfaz as mudanças
        conexao.rollback()
        print(f"Ocorreu um erro ao executar o script: {e}")
    finally:
        # Fecha o cursor
        cursor.close()


In [40]:
# Pesquisa todas as tabelas existentes no banco de dados 
def listar_tabelas_existentes(conexao):
    """Obtém todas as tabelas existentes no banco de dados no esquema 'public'."""
    cursor = conexao.cursor()
    tabelas_existentes = []
    
    try:
        cursor.execute("""
            SELECT table_name 
            FROM information_schema.tables 
            WHERE table_schema = 'public'
        """)
        tabelas_existentes = [row[0] for row in cursor.fetchall()]
    except Exception as e:
        print(f"Erro ao listar tabelas existentes: {e}")
    finally:
        cursor.close()

    return tabelas_existentes

In [41]:

# Criando tabelas no banco de acordo com o script SQL
def extrair_tabelas_do_script(caminho_arquivo_sql):
    """Extrai os nomes das tabelas do script SQL através dos comandos 'CREATE TABLE'."""
    tabelas_script = []
    padrao = re.compile(r'CREATE\s+TABLE\s+(\w+)', re.IGNORECASE)

    try:
        with open(caminho_arquivo_sql, 'r') as arquivo_sql:
            conteudo = arquivo_sql.read()
            tabelas_script = padrao.findall(conteudo)
    except Exception as e:
        print(f"Erro ao ler o arquivo SQL: {e}")

    return tabelas_script

In [71]:
conexao = conectar_banco()

In [72]:

try:
    caminho_arquivo_sql = "script_tables.sql"  
    tabelas_existentes = listar_tabelas_existentes(conexao)
    tabelas_no_script = extrair_tabelas_do_script(caminho_arquivo_sql)
    tabelas_faltantes = [tabela for tabela in tabelas_no_script if tabela not in tabelas_existentes]

    if not tabelas_faltantes:
        print("Todas as tabelas já existem. Não há necessidade de executar o script SQL.")
    else:
        print(f"Executando o script SQL, pois as seguintes tabelas ainda não existem: {', '.join(tabelas_faltantes)}")
        executar_script_sql(caminho_arquivo_sql, conexao)

except Exception as e:
    print(f"Erro ao conectar ao banco de dados: {e}")
finally:
    if conexao:
        conexao.close()

Executando o script SQL, pois as seguintes tabelas ainda não existem: curso, tipo_questao, enunciado, prova, questao, alternativas, resposta_multiplaescolha, resposta_discursiva, disciplina
Script SQL executado com sucesso.


In [73]:
# Extraindo dados do PDF e armazenando em uma string
caminho_pdf = 'provas/sistemas de informacao/2021/prova.pdf'
texto = extrair_texto_pdf(caminho_pdf)

In [74]:
texto

'3. Verifi que se a prova está completa e se o seu nome está correto no CArTÃo-reSPoSTA. Caso contrário, avise imediatamente ao Chefe de Sala.4. Assine o CArTÃo-reSPoSTA no local apropriado, com caneta esferográfi ca de ti nta preta, fabricada em material transparente.5. As respostas da prova objeti va, da prova discursiva e do questi onário de percepção da prova deverão ser transcritas, com caneta esferográfica de tinta preta, fabricada em material transparente, no CArTÃo-reSPoSTA que deverá ser entregue ao Chefe de Sala ao término da prova.6. Responda cada questão discursiva em, no máximo, 15 linhas. Qualquer texto que ultrapasse o espaço desti nado à resposta será desconsiderado.7. Você terá quatro horas para responder às questões de múlti pla escolha, às questões discursivas e ao questi onário de percepção da prova.8. Ao terminar a prova, acene para o Chefe de Sala e aguarde-o em sua carteira. Ele então irá proceder à sua identi fi cação, recolher o seu material de prova e coletar 

In [75]:
def inserir_dados(conexao, prova):
    try:
        # Cria um cursor para executar comandos SQL
        cursor = conexao.cursor()

        for item in prova:
            # Comando SQL para inserção
            sql_insert = """
            INSERT INTO prova (int_cd_curso, dat_dt_ano_aplicacao)
            VALUES (%s, %s)
            """
            cursor.execute(sql_insert, (item['id_curso'], item['ano_aplicacao']))

            conexao.commit()

            print("Dados inseridos com sucesso!")

    except Exception as e:
        print("Ocorreu um erro ao inserir os dados:", e)
        conexao.rollback()
    finally:
        cursor.close()


In [77]:
raiz = 'provas'

provas = []

conexao = conectar_banco()

for nome_materia in os.listdir(raiz):
    if nome_materia in ('sistemas de informacao', 'engenharia da computacao'):
        caminho_materia = os.path.join(raiz, nome_materia)
        
        if os.path.isdir(caminho_materia):

            for ano_aplicacao in os.listdir(caminho_materia):
                caminho_ano = os.path.join(caminho_materia, ano_aplicacao)

                if os.path.isdir(caminho_ano):

                    for arquivo in os.listdir(caminho_ano):
                        caminho_arquivo = os.path.join(caminho_ano, arquivo)

                        if os.path.isfile(caminho_arquivo) and arquivo.lower().endswith('.pdf') and arquivo.startswith('prova'):
                            id_materia = pesquisar_materia(conexao, nome_materia)
                            
                            provas.append({
                                'nome_curso': nome_materia,
                                'id_curso': id_materia,
                                'ano_aplicacao': ano_aplicacao,
                                'arquivo': arquivo
                            })

                            print(f"Matéria: {nome_materia}, Id_Materia: {id_materia}, Ano: {ano_aplicacao}, Prova: {arquivo} ")


inserir_dados(conexao, provas)


Matéria: engenharia da computacao, Id_Materia: 1, Ano: 2014, Prova: prova.pdf 
Matéria: engenharia da computacao, Id_Materia: 1, Ano: 2017, Prova: prova.pdf 
Matéria: engenharia da computacao, Id_Materia: 1, Ano: 2019, Prova: prova.pdf 
Matéria: engenharia da computacao, Id_Materia: 1, Ano: 2023, Prova: prova.pdf 
Matéria: sistemas de informacao, Id_Materia: 2, Ano: 2017, Prova: prova.pdf 
Matéria: sistemas de informacao, Id_Materia: 2, Ano: 2021, Prova: prova.pdf 
Dados inseridos com sucesso!
Dados inseridos com sucesso!
Dados inseridos com sucesso!
Dados inseridos com sucesso!
Dados inseridos com sucesso!
Dados inseridos com sucesso!


In [78]:
questos_discursivas = salvar_questoes_discursivas(texto)
print(questos_discursivas)

[{'numero': 'QUEStãO diSCURSiVa 01', 'enunciado': 'tEXtO i fORMaçãO GERaL fORMaçãO GERaL Em época de censura, a própria existência da arte passa a ser questi onada. Surgem debates em jornais, na rua, em casa, para discuti r sua relevância. Não podemos deixar de nos perguntar como chegamos a essa estranha situação  em que  precisamos  justi fi car a própria existência  da arte. Ela pode  ser julgada apressadamente como boa ou ruim, mas nem por isso deixa de ser arte. O cineasta franco-suíço Jean-Luc Godard aponta para o fato de que “a cultura é a regra; a arte é a exceção”. A  arte  é,  dentro  da  cultura,  o  que  tensiona  a  própria  cultura  para  assim  levá-la  para  outros  lugares. Enquanto a cultura regula, a arte destoa e movimenta. A arte questi ona, incomoda e transforma. Arte e cultura se contradizem, mas andam de mãos dadas. Os psicanalistas Suely Rolnik e Félix Guatt ari consideram que o conceito de cultura é profundamente reacionário.  É  uma  maneira  de  separar  ati 

In [79]:
questoes = salvar_questoes(texto)
for questao in questoes:
    print(questao)

{'numero': 'QUEStãO 01', 'enunciado': 'A chance de uma criança de baixa renda ter um futuro melhor que a realidade em que nasceu está, em maior ou menor grau, relacionada à escolaridade e ao nível de renda de seus pais. Nos países ricos, o "elevador social" anda mais rápido. Nos emergentes, mais devagar. No Brasil, ainda mais lentamente. O  país  ocupa  a  segunda  pior  posição  em  um  estudo  sobre  mobilidade  social  feito  pela  Organização para a Cooperação e Desenvolvimento Econômico (OCDE), em 2018, com dados de 30 países. Segundo os resultados, seriam necessárias nove gerações para que os descendentes de um brasileiro entre os 10%  mais  pobres  ati ngissem  o  nível  médio  de  rendimento  do  país.  A  esti mati va  é  a  mesma  para  a África  do  Sul  e  só  perde  para  a  Colômbia,  onde  o  período  de  ascensão  levaria  11  gerações.  Mais  de 1/3 daqueles que nascem entre os 20% mais pobres no Brasil permanece na base da pirâmide, enquanto apenas  7%  consegue  cheg

In [80]:
for i in range(0, len(questoes)):
    print(f"Questão {questoes[i]['numero']}")
    print(f"Enunciado: {questoes[i]['enunciado']}")
    print(f"Alternativas:")
    for letra, alternativa in questoes[i]['alternativas'].items():
        print(f"{letra}) {alternativa}")

Questão QUEStãO 01
Enunciado: A chance de uma criança de baixa renda ter um futuro melhor que a realidade em que nasceu está, em maior ou menor grau, relacionada à escolaridade e ao nível de renda de seus pais. Nos países ricos, o "elevador social" anda mais rápido. Nos emergentes, mais devagar. No Brasil, ainda mais lentamente. O  país  ocupa  a  segunda  pior  posição  em  um  estudo  sobre  mobilidade  social  feito  pela  Organização para a Cooperação e Desenvolvimento Econômico (OCDE), em 2018, com dados de 30 países. Segundo os resultados, seriam necessárias nove gerações para que os descendentes de um brasileiro entre os 10%  mais  pobres  ati ngissem  o  nível  médio  de  rendimento  do  país.  A  esti mati va  é  a  mesma  para  a África  do  Sul  e  só  perde  para  a  Colômbia,  onde  o  período  de  ascensão  levaria  11  gerações.  Mais  de 1/3 daqueles que nascem entre os 20% mais pobres no Brasil permanece na base da pirâmide, enquanto apenas  7%  consegue  chegar  aos  

In [81]:
def pesquisar_prova(conexao, nome_materia, ano_aplicacao):
    try:
        with conexao.cursor() as cursor:
            query = """
            SELECT int_cd_prova
            FROM prova
            LEFT JOIN curso ON prova.int_cd_curso = curso.int_cd_curso
            WHERE curso.chr_curso = %s AND prova.dat_dt_ano_aplicacao = %s
            """
            
            cursor.execute(query, (nome_materia, ano_aplicacao))
            
            resultados = cursor.fetchall()
            
            if resultados:
                return resultados[0][0]  
            else:
                print("Nenhuma prova encontrada.")
    
    except Exception as e:
        print(f"Erro ao executar a pesquisa: {e}")


In [82]:
conexao = conectar_banco()
id_prova = pesquisar_prova(conexao, 'sistemas de informacao', '2021')
id_tipo_questao = 2  # 1 para discursiva e 2 para mulitpla escolha

In [84]:
try:
    cursor = conexao.cursor()
    for i in range(1, len(questoes)):
        if questoes[i]['numero'].lower() not in ('questão 11', 'questão 13', 'questao 23', 'questao 26'):

            sql = """INSERT INTO enunciado (chr_enunciado)"""
            sql += """VALUES (%s) RETURNING int_cd_enunciado"""
            cursor.execute(sql, (questoes[i]['enunciado'],))
            id_enunciado = cursor.fetchone()[0]

            sql = """INSERT INTO questao (int_cd_prova, int_cd_tipo_questao, int_cd_enunciado, chr_titulo_questao)"""
            sql += """VALUES (%s, %s, %s, %s) RETURNING int_cd_questao"""
            cursor.execute(sql, (id_prova, id_tipo_questao, id_enunciado, questoes[i]['numero'].lower())) 
            id_questao = cursor.fetchone()[0]

            for letra, alternativa in questoes[i]['alternativas'].items():
                padrao = r'^[A-Z]\s+'
                alternativa = re.sub(padrao, '', alternativa)
                sql = """INSERT INTO alternativas (chr_alternativa, int_cd_questao, chr_letra_alternativa)"""
                sql += """VALUES (%s, %s, %s)"""
                cursor.execute(sql, (alternativa, id_questao, letra))

    conexao.commit()
    print("Dados inseridos com sucesso!")

except Exception as e:
    print("Ocorreu um erro ao inserir os dados:", e)
    conexao.rollback()


Dados inseridos com sucesso!


In [85]:
# armazenando questões discursivas
id_tipo_questao = 1  # 1 para discursiva e 2 para objetiva
try:
    cursor = conexao.cursor()
    for i in range(0, len(questos_discursivas)):
        sql = """INSERT INTO enunciado (chr_enunciado)"""
        sql += """VALUES (%s) RETURNING int_cd_enunciado"""
        cursor.execute(sql, (questos_discursivas[i]['enunciado'],))
        id_enunciado = cursor.fetchone()[0]

        sql = """INSERT INTO questao (int_cd_prova, int_cd_tipo_questao, int_cd_enunciado, chr_titulo_questao)"""
        sql += """VALUES (%s, %s, %s, %s) RETURNING int_cd_questao"""
        cursor.execute(sql, (id_prova, id_tipo_questao, id_enunciado, questos_discursivas[i]['numero'].lower()))
        id_questao = cursor.fetchone()[0]

    conexao.commit()
    print("Dados inseridos com sucesso!")

except Exception as e:
    print("Ocorreu um erro ao inserir os dados:", e)
    conexao.rollback()


Dados inseridos com sucesso!


In [86]:
caminho_respostas = 'provas/sistemas de informacao/2021/gabarito.pdf'

In [87]:
respostas = extrair_texto_pdf(caminho_respostas)

In [88]:
# Texto lido do PDF, armazenado como uma lista de strings (simulação do .split('\n'))
texto = respostas.split('\n')

# Variáveis de controle
texto_limpo = []
capturando_apos_gabarito = False

# Iterar sobre o texto, começando a capturar a partir do valor 'Gabarito'
for item in texto:
    item = item.strip()  # Remover espaços em branco
    
    if item == "Gabarito":
        capturando_apos_gabarito = True  # Começar a capturar após encontrar 'Gabarito'
        continue  # Pular o item "Gabarito" e ir para o próximo
    
    if capturando_apos_gabarito and item != '':  # Ignorar valores vazios
        texto_limpo.append(item)

# Exibir a lista limpa
print(texto_limpo)


['QUESTÃO DISCURSIVA 1', 'QUESTÃO DISCURSIVA 2', 'QUESTÃO 1', 'QUESTÃO 2', 'QUESTÃO 3', 'QUESTÃO 4', 'QUESTÃO 5', 'QUESTÃO 6', 'QUESTÃO 7', 'QUESTÃO 8', 'QUESTÃO DISCURSIVA 3', 'QUESTÃO DISCURSIVA 4', 'QUESTÃO DISCURSIVA 5', 'QUESTÃO 9', 'QUESTÃO 10', 'QUESTÃO 11', 'QUESTÃO 12', 'QUESTÃO 13', 'QUESTÃO 14', 'QUESTÃO 15', 'QUESTÃO 16', 'QUESTÃO 17', 'QUESTÃO 18', 'QUESTÃO 19', 'QUESTÃO 20', 'QUESTÃO 21', 'QUESTÃO 22', 'QUESTÃO 23', 'QUESTÃO 24', 'QUESTÃO 25', 'QUESTÃO 26', 'QUESTÃO 27', 'QUESTÃO 28', 'QUESTÃO 29', 'QUESTÃO 30', 'QUESTÃO 31', 'QUESTÃO 32', 'QUESTÃO 33', 'QUESTÃO 34', 'QUESTÃO 35', '***', '***', 'E', 'C', 'B', 'B', 'A', 'A', 'C', 'D', '***', '***', '***', 'C', 'E', 'C', 'B', 'D', 'C', 'D', 'A', 'C', 'B', 'A', 'B', 'C', 'E', 'B', 'B', 'E', 'E', 'D', 'A', 'D', 'E', 'A', 'C', 'ANULADA', 'D', 'D']


In [89]:
# Lista de questões e respostas
dados = texto_limpo

# Separar as questões das respostas
questoes = []
respostas = []
is_question_part = True

for item in dados:
    if "QUESTÃO" in item:  # Identifica as questões
        questoes.append(item)
        is_question_part = False  # A partir desse ponto as respostas começam
    elif not is_question_part:
        respostas.append(item)  # Armazena as respostas após as questões

# Função para formatar o número da questão
def formatar_questao(questao):
    partes = questao.split()
    # Verifica se o último elemento é um número
    if partes[-1].isdigit():  # Para questões numéricas e discursivas
        numero = int(partes[-1])  # Pega o número
        if "DISCURSIVA" in questao:  # Se for uma questão discursiva
            return f'questão discursiva {numero:02d}'  # Formata como discursiva
        else:
            return f'questão {numero:02d}'  # Formata como questão normal
    return questao.lower()  # Para outros casos

# Mapeando questões e suas respectivas respostas
questoes_e_respostas = {}

for i, questao in enumerate(questoes):
    questao_formatada = formatar_questao(questao)  # Formatando a questão
    resposta = respostas[i] if i < len(respostas) else None
    questoes_e_respostas[questao_formatada] = resposta

# Exibir questões e respostas associadas
for questao, resposta in questoes_e_respostas.items():
    print(f'{questao}: {resposta}')


questão discursiva 01: ***
questão discursiva 02: ***
questão 01: E
questão 02: C
questão 03: B
questão 04: B
questão 05: A
questão 06: A
questão 07: C
questão 08: D
questão discursiva 03: ***
questão discursiva 04: ***
questão discursiva 05: ***
questão 09: C
questão 10: E
questão 11: C
questão 12: B
questão 13: D
questão 14: C
questão 15: D
questão 16: A
questão 17: C
questão 18: B
questão 19: A
questão 20: B
questão 21: C
questão 22: E
questão 23: B
questão 24: B
questão 25: E
questão 26: E
questão 27: D
questão 28: A
questão 29: D
questão 30: E
questão 31: A
questão 32: C
questão 33: ANULADA
questão 34: D
questão 35: D


In [90]:
for questao, resposta in questoes_e_respostas.items():
    if not questao.startswith('questão discursiva'):
        print(questao, resposta)

questão 01 E
questão 02 C
questão 03 B
questão 04 B
questão 05 A
questão 06 A
questão 07 C
questão 08 D
questão 09 C
questão 10 E
questão 11 C
questão 12 B
questão 13 D
questão 14 C
questão 15 D
questão 16 A
questão 17 C
questão 18 B
questão 19 A
questão 20 B
questão 21 C
questão 22 E
questão 23 B
questão 24 B
questão 25 E
questão 26 E
questão 27 D
questão 28 A
questão 29 D
questão 30 E
questão 31 A
questão 32 C
questão 33 ANULADA
questão 34 D
questão 35 D


In [93]:
# Salvar respostas das questões
id_prova = pesquisar_prova(conexao, 'sistemas de informacao', '2021')

try:
    cursor = conexao.cursor()
    for questao, resposta in questoes_e_respostas.items():
        # Questão não começar com questao discursiva
        if not questao.startswith('questão discursiva'):
            sql = """SELECT int_cd_questao FROM questao WHERE chr_titulo_questao = %s AND int_cd_prova = %s"""
            cursor.execute(sql, (questao, id_prova))

            result_questao = cursor.fetchone()


            if result_questao is None:
                print(f"Questão '{questao}' não encontrada.")
                continue

            id_questao = result_questao[0]

            sql = """SELECT int_cd_alternativa FROM alternativas WHERE chr_letra_alternativa = %s AND int_cd_questao = %s"""
            cursor.execute(sql, (resposta, id_questao))
            result_alternativa = cursor.fetchone()

            if result_alternativa is None:
                print(f"Alternativa '{resposta}' para a questão '{questao}' não encontrada.")
                continue
            
            id_alternativa = result_alternativa[0]

            sql = """INSERT INTO resposta_multiplaescolha (int_cd_questao, int_cd_alternativa)"""
            sql += """VALUES (%s, %s)"""
            cursor.execute(sql, (id_questao, id_alternativa))

    conexao.commit()
    print("Dados inseridos com sucesso!")

except Exception as e:
    print("Ocorreu um erro ao inserir os dados:", e)
    conexao.rollback()

Questão 'questão 01' não encontrada.
Questão 'questão 11' não encontrada.
Questão 'questão 12' não encontrada.
Questão 'questão 13' não encontrada.
Questão 'questão 14' não encontrada.
Questão 'questão 24' não encontrada.
Questão 'questão 25' não encontrada.
Questão 'questão 32' não encontrada.
Alternativa 'ANULADA' para a questão 'questão 33' não encontrada.
Questão 'questão 34' não encontrada.
Dados inseridos com sucesso!


In [94]:
arquivo_padrao_resposta = 'provas/sistemas de informacao/2021/padrao_resposta.pdf'

padrao_resposta = extrair_texto_pdf(arquivo_padrao_resposta)

# Texto lido do PDF, armazenado como uma lista de strings (simulação do .split('\n'))
texto = padrao_resposta.split('\n')

texto_limpo = []
for item in texto:
    item = item.strip()
    if item != '':
        texto_limpo.append(item)

print(texto_limpo)


['1', '2', '|', 'O', 'R', 'B', 'M', 'E', 'V', 'O', 'N', 'SISTEMAS DE INFORMAÇÃO', 'Bacharelado', '27', 'QUESTÃO DISCURSIVA 1', 'TEXTO I', 'Em época de censura, a própria existência da arte passa a ser questionada. Surgem debates em', 'jornais, na rua, em casa, para discutir sua relevância. Não podemos deixar de nos perguntar como', 'chegamos a essa estranha situação em que precisamos justificar a própria existência da arte. Ela', 'pode ser julgada apressadamente como boa ou ruim, mas nem por isso deixa de ser arte.', 'O cineasta franco-suíço Jean-Luc Godard aponta para o fato de que “a cultura é a regra; a arte é', 'a  exceção”.  A  arte  é,  dentro  da  cultura,  o  que  tensiona  a  própria  cultura  para  assim  levá-la  para', 'outros lugares. Enquanto a cultura regula, a arte destoa e movimenta. A arte questiona, incomoda e', 'transforma. Arte e cultura se contradizem, mas andam de mãos dadas.', 'Os psicanalistas Suely Rolnik e Félix Guattari consideram que o conceito de cultura é

In [95]:
dados = texto_limpo

questoes = []
padrao_resposta = []
coletando_respostas = False  # Controla se estamos no bloco de respostas
resposta_temp = ""  # Armazena temporariamente as respostas

for item in dados:
    if "QUESTÃO" in item:  # Identifica a questão
        questoes.append(item)
        coletando_respostas = False  # Paramos de coletar respostas temporariamente
    elif "PADRÃO DE RESPOSTA" in item:  # Identifica o padrão de resposta
        coletando_respostas = True  # Começamos a coletar o bloco de respostas
        if resposta_temp:  # Se já houver uma resposta temporária, salvamos
            padrao_resposta.append(resposta_temp.strip())  # Remove espaços extras
            resposta_temp = ""  # Reinicializa a resposta temporária
    elif coletando_respostas:
        resposta_temp += item + " "  # Continuamos coletando a resposta

# Certifica-se de adicionar a última resposta ao final
if resposta_temp:
    padrao_resposta.append(resposta_temp.strip())

# Função para formatar o número da questão
def formatar_questao(questao):
    partes = questao.split()
    if partes[-1].isdigit():  # Verifica se o último elemento é um número
        numero = int(partes[-1])
        if "DISCURSIVA" in questao:
            return f'questão discursiva {numero:02d}'
    return questao.lower()

# Mapeando questões e padrões de resposta
questoes_e_padrao_resposta = {}

for i, questao in enumerate(questoes):
    questao_formatada = formatar_questao(questao)
    resposta = padrao_resposta[i] if i < len(padrao_resposta) else None
    questoes_e_padrao_resposta[questao_formatada] = resposta

# Exibindo as questões e seus respectivos padrões de resposta
for questao, resposta in questoes_e_padrao_resposta.items():
    print(f'{questao}: {resposta}')


questão discursiva 01: O respondente deve, a partir dos argumentos presentes no texto I, refletir sobre as tensões existentes entre a arte e a cultura no Brasil contemporâneo e sobre a liberdade artística explicitado no artigo 5º da Constituição Federal (Texto II), de modo a perceber a ilegitimidade dos movimentos de censura que tem eclodido em determinados segmentos da sociedade brasileira. O respondente deve, ainda, apresentar duas ações educativas para a superação das tensões citadas, como: encontros de artistas e público em escolas e outros espaços públicos; projetos de visitação a espaços culturais, como museus e galerias, voltados para a formação de público/plateia; debates em espaços públicos a respeito da liberdade artística, etc. (Valor: 10,0 pontos)
questão discursiva 02: a)   O respondente deve mencionar que as cidades inteligentes podem diminuir o impacto ambiental dos  aglomerados urbanos, pois, ao utilizarem a tecnologia como um fator indispensável  para modernizar e ofer

In [97]:
conexao = conectar_banco()
id_prova = pesquisar_prova(conexao, 'sistemas de informacao', '2021')

try:
    cursor = conexao.cursor()
    for questao, resposta in questoes_e_padrao_resposta.items():
        if questao.startswith('questão discursiva'):
            sql = """SELECT int_cd_questao FROM questao WHERE chr_titulo_questao = %s AND int_cd_prova = %s"""
            cursor.execute(sql, (questao, id_prova))
            result_questao = cursor.fetchone()

            if result_questao is None:
                print(f"Questão '{questao}' não encontrada.")
                continue

            id_questao = result_questao[0]

            sql = """INSERT INTO resposta_discursiva (int_cd_questao, txt_padrao_resposta)"""
            sql += """VALUES (%s, %s)"""
            cursor.execute(sql, (id_questao, resposta))

    conexao.commit()
    print("Dados inseridos com sucesso!")
except Exception as e:
    print("Ocorreu um erro ao inserir os dados:", e)
    conexao.rollback()


Dados inseridos com sucesso!
